<a href="https://colab.research.google.com/github/ashi-sharmaa/UTAustinCamp-NLPModules/blob/main/UTAustinSummerCampNLPModules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the second NLP hands-on module!

*IMPORTANT*: You should first *copy* this notebook to your Google drive.
---
This module have two exercises (1) play with n-gram language model and (2) play with APIs of state-of-the-art langauge models. Let's get started with loading datasets and libraries like before!

** credit: n-gram part is edited from UMass Amherst homework (CS585 HW2, Mohit Iyyer)**





# Step 0-1: Loading Datasets
**Please run CELLS in the order they are presented!**

In [ ]:
! pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pickle, os, sys, random, time, re, math, operator
import matplotlib.pyplot as plt
from collections import *
import numpy as np
from numpy import dot
from numpy.linalg import norm
from datasets import load_dataset
from transformers import BertTokenizer
subword_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
task1_data = load_dataset('tweet_eval', 'irony')
task2_data = load_dataset('gsm8k', 'main')
task3_data = load_dataset('eli5_category')
lm_dataset = load_dataset('wikitext', 'wikitext-103-v1')
lm_dataset = lm_dataset['train'].filter(lambda e,i: i<20000, with_indices=True)

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2862 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/784 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/955 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/irony/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


...

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Filter:   0%|          | 0/1801350 [00:00<?, ? examples/s]

# Part 1: N-gram Language Model

### 1-1 Training a language model

We provide an implementation of a very simple n-gram language model here. The function will take sentences from Wikipedia corpus, and output a dictionary where the key is the history and the value is a probability distribution over the next word over the vocabulary.

Each key should be a single string where the words that form the history have been concatenated using spaces. Given a key, its corresponding value should be a dictionary where each word type in the vocabulary is associated with its probability of appearing after the key. For example, the entry for the history 'w1 w2', modeling P(w3 |w1, w2) should look like:

    lm['w1 w2'] = {'w0': 0.001, 'w1' : 2e-6, 'w2' : 1e-6, 'w3': 1e-9, ...}

We will also have an entry for empty history, capturing probability of words in the corpus.

    lm[''] = {'w0': 0.01, 'w1' : 3e-6, 'w2' : 4e-6, 'w3': 0.0009, ...}


Let's train the language model now! This will take some time (<5 minutes).

In [ ]:
def train_ngram_lm(data):
    lm = defaultdict(Counter)
    count = 0
    for example in data:
      count += 1
      tokenized_sentence = subword_tokenizer.tokenize('[CLS] '+example['text']+' [SEP]')
      for i in range(1, len(tokenized_sentence)):
        # increment trigram: count(x_{i-2}, x_{i-1}, x_i)
        if i > 2:
          lm[tokenized_sentence[i-2] + " " + tokenized_sentence[i-1]].update([tokenized_sentence[i]])
        # increment bigram: count(x_{i-1}, x_i)
        lm[tokenized_sentence[i-1]].update([tokenized_sentence[i]])
        # no context, increment unigram: count (x_i)
        lm[''].update([tokenized_sentence[i]])
      if count%2000==0:
        print('%d/20000 lines processed so far.' % count)
    # normalize to map counts into a probability distribution
    for _, v in lm.items():
      total = sum(v.values(), 0.0)
      for key in v:
        v[key] /= total
    return lm
# We train our model here!
lm = train_ngram_lm(lm_dataset)

2000/20000 lines processed so far.
4000/20000 lines processed so far.
6000/20000 lines processed so far.
8000/20000 lines processed so far.
10000/20000 lines processed so far.
12000/20000 lines processed so far.
14000/20000 lines processed so far.
16000/20000 lines processed so far.
18000/20000 lines processed so far.
20000/20000 lines processed so far.


## Examine probability distribution from different contexts!

Running the following codes will output top 10 most commonly occurring next word given the provided context. '' means empty context, thus providing the most common words in the dataset. if you provide "the" as a context, then it'll give top 10 most likely follow up words after "the".

In [ ]:
lm[''].most_common(10)

[('the', 0.05567690762291043),
 (',', 0.04388812023805011),
 ('.', 0.03528818088776013),
 ('of', 0.024124843637466357),
 ('and', 0.021690168429298863),
 ('@', 0.020011245467065945),
 ('in', 0.019476611955574086),
 ('to', 0.016898209570018828),
 ('[SEP]', 0.015794195317336973),
 ('a', 0.015350378428919804)]

In [ ]:
lm['the'].most_common(10)

[('first', 0.016864530587350893),
 ('song', 0.007928740620966483),
 ('album', 0.007162815766704963),
 ('game', 0.006850772307561381),
 ('united', 0.006808220926769074),
 ('same', 0.006652199197197282),
 ('"', 0.006439442293235749),
 ('city', 0.006155766421287037),
 ('new', 0.006056479866104988),
 ('second', 0.006028112278910118)]

In [ ]:
lm['the best'].most_common(10)

[('@', 0.13043478260869565),
 ('of', 0.062111801242236024),
 ('song', 0.037267080745341616),
 ('known', 0.031055900621118012),
 ('in', 0.024844720496894408),
 ('preserved', 0.018633540372670808),
 ('chance', 0.012422360248447204),
 ('defensive', 0.012422360248447204),
 ('man', 0.012422360248447204),
 ('##sell', 0.012422360248447204)]

In [ ]:
lm['walked'].most_common(10)

[('around', 0.125),
 ('the', 0.125),
 ('in', 0.125),
 ('to', 0.125),
 ('onto', 0.125),
 ('offs', 0.125),
 ('out', 0.125),
 ('up', 0.125)]

### 1-2 Generate text from n-gram language model

We have an implementation of a function (generate_text) which will generate continuation of context according to the language model we just trained.

*Input:*
+ **lm**: the lm object, a dictionary you return from  the **train_ngram_lm** function
+ **context**: the input context string that you want to condition your language model on, should be a space-separated string of tokens
+ **order**: order of your language model (i.e., "n" in the n-gram model). We support up to trigram.
+ **num_tok**: number of tokens to be generated following the input context (or until the stop token ([SEP]) is generated.

*Output:*
+ a list of generated tokens (strings).



 ### Sampling from distribution

 We will use **[numpy.random.choice](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.choice.html)** to sample the next word from the distribution. Let's see how this function works a bit here!
 It will take a list of options (first argument) and probability distribution over the options (third argument) and provide sampled set of provided size (second argument). Try changing probability distribution and see sampling behaviors! Run the code multiple times to see if predictions change. Why or why not?



In [ ]:
np.random.choice(['summer','camp','texas'], size=7, p=[0.6,0.1,0.3])

array(['texas', 'camp', 'texas', 'summer', 'summer', 'texas', 'summer'],
      dtype='<U6')

In [ ]:
def generate_text(lm, context="he is the", order=3, num_tok=20):
    # The goal is to generate new words following the context
    # If context has more tokens than the order of lm,
    # generate text that follows the last (order-1) tokens of the context
    # and store it in the variable `history`
    order -= 1
    # tokenize the context
    tokenized_context = subword_tokenizer.tokenize(context)
    if order == 0:
      history = ''
    else:
      history = tokenized_context[-order:]
    # `out` is the list of tokens of context
    out = tokenized_context
    for i in range(num_tok):
      # default is unigram distribution
      distribution = lm[''].items()
      history_txt = ' '.join(history)
      # if bigram is in lm dictionary, get its distribution
      if history_txt in lm:
        distribution = lm[history_txt].items()
      elif order > 1:
        # if not, take unigram distribution
        history_txt = ' '.join(history[-(order+1):])
        if history_txt in lm:
          distribution = lm[history_txt].items()
      generated_token = np.random.choice([k for k, _ in distribution], p=[v for _, v in distribution])
      out.append(generated_token)
      # if generated end token, finish generation here.
      if generated_token == '[SEP]':
        return out
      # otherwise, keep generating after updating the history.
      if history:
        history = history[1:] + [generated_token]
    return out

With order set to 1, we will generate from the unigram langauge model. This will sample each word independently until you sample end of sequence token.

In [ ]:
order = 1
outputs = generate_text(lm, context='summer is', order=order)
print(outputs)

['summer', 'is', 'the', 'to', 'molecule', 'the', 'last', 'biographer', 'that', 'german', 'finished', 'three', 'roller', 'the', 'the', 'do', 'kingdom', 'memorial', '@', 'after', 'landings', '.']


With order set to 2, we will generate from bigram language model. Each word is sampled from  a probability distribution of next token based on its one previous word.

In [ ]:
order = 2
outputs = generate_text(lm, context='summer is', order=order)
print(outputs)

['summer', 'is', 'interpreted', 'by', 'the', 'gerry', 'lane', '.', 'mon', '##ym', 'sought', 'after', 'overcoming', 'anything', 'for', 'the', 'nintendo', 'ds', 'lit', '##iga', '##tors', ',']


With order set to 3, we will generate from trigram language model. Each word is sampled from a probability distribution of next token based on its two previous words.

In [ ]:
order = 3
outputs = generate_text(lm, context='summer is', order=order)
print(outputs)

['summer', 'is', 'site', 'initially', '@', '-', '@', 'star', 'game', ',', 'de', '##gr', '##om', 'is', 'known', 'as', 'cr', '##ac', 'des', 'chevalier', '##s', 'was']


### Q1. How would you characterize output from unigram, bigram, and trigram models?

Change provided context to generate sentences from different prefixes. Describe what you observe. When you change context, does it impact the generation of unigram model (generating with order 1)? Does it impact the generation of bigram (generation with order 2) or trigram model (generation with order 3)?

### Q1 Answer: [TODO]


# Part 2: Play with large LLM interface.

Our main goal is to understand the weaknesses and strength of modern large-scale language models. You probably already have experiences interacting with these models! This exercise is designed to help you understand the limitations and promises of these model. The colab notebook *only* provides you examples that you would test out. All interaction with large language model will happen at public LM API demos, which you can log in from Step 2-1.


# Step 2-1: Assessing public LM demos

Please log into [Claude](https://poe.com/Dragonfly) or [ChatGPT](https://chat.openai.com/auth/login). You should use their free version and will be required to create an account, if you don't have it already.






## Section 2-2: Binary Classification Task
Here we will ask language model to classify each tweet whether it is ironic or not. In this dataset, ironic sentence will be tagged with label 1, and non-ironic sentence will be taged with label 0. You can print some data below.

In [ ]:
task1_eval = task1_data['test'].filter(lambda e,i: i<20, with_indices=True)

Filter:   0%|          | 0/784 [00:00<?, ? examples/s]

In [ ]:
for example in task1_eval:
  print(example)

{'text': '@user Can U Help?||More conservatives needed on #TSU + get paid 4 posting stuff like this!||YOU $ can go to', 'label': 0}
{'text': 'Just walked in to #Starbucks and asked for a "tall blonde" Hahahaha #irony', 'label': 1}
{'text': '#NOT GONNA WIN', 'label': 0}
{'text': '@user He is exactly that sort of person. Weirdo!', 'label': 0}
{'text': "So much #sarcasm at work mate 10/10 #boring 100% #dead mate full on #shit absolutely #sleeping mate can't handle the #sarcasm", 'label': 1}
{'text': 'Corny jokes are my absolute favorite', 'label': 0}
{'text': 'People complain about my backround pic and all I feel is like "hey don\'t blame me, Albert E might have spoken those words" #sarcasm #life', 'label': 1}
{'text': '@user @user Darn, my sock joke needs fixing?', 'label': 0}
{'text': "if Christian expects Fifa to sleep in my bed with me tonight, he's wrong 👿", 'label': 0}
{'text': 'People who tell people with anxiety to "just stop worrying about it" are my favorite kind of people #not 

Now, you will interact with language models to *prompt* it to generate an output label given an input text. You should change "Sentence" to sentence you would like to try (here I am using the third example), and observe the model output.

## *Q1* Simple Prompt

Take the example tweets from the test set printed above and provide it as an {{INPUT}} to the following simplest prompt. Try a few examples. How does model answer?
```
Is this sentence ironic? {{INPUT}}
```


### Q1 Answer: [TODO]


## Q2 Simple prompt with output formats:

You now ask specifically for a yes or no answer (you can tweak your prompt format). What does model do?


```
Is this sentence ironic? Answer yes or no. {{INPUT}}
```



### Q2 Answer: [TODO]

## Section 2-2: Mathematical Reasoning Task
Now we will look at slightly complex task, of math problem solving! Language model is mostly designed to understand natural language, not specialized for mathematical reasoning. But could it do it well? We will examine this question here.

We will look at examples from GSM8K (Grade School Math 8K) is a dataset high quality linguistically diverse grade school math word problems. The dataset was created to support the task of question answering on basic mathematical problems that require multi-step reasoning.

Increase the number "10" below, if you'd like to see more examples.

In [ ]:
task2_eval = task2_data['test'].filter(lambda e,i: i<10, with_indices=True)

Filter:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
for example in task2_eval:
  print('---')
  answers = example['answer'].split("####")
  rationale = answers[0]
  answer_value = answers[1]
  print("* Quesition: " + example['question'])
  print("* Rationale: " + rationale)
  print("* Final Answer: " + answer_value)

---
* Quesition: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
* Rationale: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.

* Final Answer:  18
---
* Quesition: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
* Rationale: It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric

* Final Answer:  3
---
* Quesition: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
* Rationale: The cost of the house and repairs came out to 80,000+50,000=$<<80000+5

### Q4: Math problem solving with language model

You can use simple prompt like the following, where Input is the question. Try a few examples.

```
Question: {{Input}}
```

Do they get the answer correctly? Do they provide rationale? Do you see instances where the answer (or reasoning process) is incorrect?

### Q4 Answer: [TODO]

### Section 2-3: Long form question answering
Now we will look at knowledge intensive task, where we pose questions (those posted on reddit [ELI5 forum](https://www.reddit.com/r/explainlikeimfive/) (explain like i'm five) previously) to language models. These are often complex questions that requires multi sentence answers and somewhat ambiguous. Let's look at examples first!

In [ ]:
task3_eval = task3_data['test'].filter(lambda e,i: i<10, with_indices=True)

In [ ]:
for example in task3_eval:
  print('---')
  print("* Quesition: " + example['title'])
  for answer in example['answers']['text']:
    print("\t* Answer: "+ answer)

---
* Quesition: Why do old games running on new hardware still have technical issues?
	* Answer: The XBox is emulating NES hardware and running the emulation at a set speed. If it ran it at as fast as possible, then it would be several times faster than the original NES game and would be unplayable. I can't speak for Mega Man exactly, but older games tended to run on a cycle locked to the screen refresh which was a fixed 60Hz or 50Hz. There was only one piece of hardware they ran on, so there was no need to adjust for different hardware speeds.
	* Answer: In that case, it's probably on purpose - they want to emulate the experience as closely as possible, even including the slowdown and sprite flickering. Some emulators let you turn it off, but it's usually turned on by default. In other cases, like if you're trying to emulate PS2 games on your PC, the game might just run really slow in general. Even though your PC is way more powerful than a PS2, it has to "translate" from PS2 languag

### Q5: Long form QA with language model.

You can re use simple prompt like the following, where Input is the question. Try a few examples.

```
Question: {{Input}}
```

Could you evaluate the answer or compare it to the reference human-written answers?

###Q5 Answer: [TODO]


### Section 4: Further limitations of LLMs

Here are some simple questions/exercises where LLMs will have a hard time answering correctly.


1. Models have a limited ability in engaging with physical properties or common sense. Try few examples below! Why would models make such mistakes? Try changing the prompt to help the model.
```
sort the following animals in the order of typical size: cat, dog, lizard, platypus, bonobo, crocodile, parrot, whale, dolphin
```


2. Commands that involve the shape of the words.
```
politicians from Texas whose first name is seven characters.
```


3. Highly compositional queries which potentially seek an empty set.
```
name an actress who is born in germany and received an oscar award at least once.
```

4. Ask the same question multiple times, with slightly different prompt. Does model give consistent answer? Or does it contradict itself?

### Q6 Answer [TODO]: Describe your findings here.
